In [1]:
import ee
import pandas as pd
import time
import datetime
import geemap
import pprint
import numpy as np

Map = geemap.Map(height='800px')

pp = pprint.PrettyPrinter(depth=4)

In [2]:
ee.Authenticate()

Enter verification code: 4/1AfDhmrg4A7kK78XLH6bx4CAsX2jtHe79EMdiDkdSgy4YRVlt5-SAf4hlN2Q

Successfully saved authorization token.


In [3]:
try:    
    ee.Initialize()
    print('Google Earth Engine has initialized successfully!')
except ee.EEException as e:
    print('Google Earth Engine has failed to initialize!')
except:
    print("Unexpected error:", sys.exc_info()[0])
    raise

Google Earth Engine has initialized successfully!


In [4]:
modisLandcover = ee.ImageCollection("MODIS/006/MCD12Q1")
filtered = modisLandcover.filter(ee.Filter.date('2018-01-01', '2018-12-31'))
landcover2018 = ee.Image(filtered.first())
classified = landcover2018.select('LC_Type1')

In [6]:
palette = ['05450a', '086a10', '54a708',
                 '78d203', '009900', 'c6b044','dcd159', 
                 'dade48', 'fbff13', 'b6ff05', '27ff87',
                 'c24f44', 'a5a5a5', 'ff6d4c', '69fff8',
                 'f9ffa4', '1c0dff']

Map.addLayer(classified, {'min':1, 'max':17, 'palette': palette}, 'MODIS Landcover 2018',0)
#Map

In [7]:
gaul = ee.FeatureCollection('FAO/GAUL_SIMPLIFIED_500m/2015/level2')
kerala = gaul.filter(ee.Filter.eq('ADM1_NAME', 'Kerala'))
Map.addLayer(kerala, {'color': 'purple'}, 'Admin2 Boundaries')
Map.centerObject(kerala,8)
#Map

In [8]:
keralaLandcover = classified.clip(kerala)
Map.addLayer(keralaLandcover, {'min':1, 'max':17, 'palette': palette}, 'Kerala Land Cover 2018')
Map

Map(center=[10.448735297125276, 76.40719109837225], controls=(WidgetControl(options=['position'], widget=HBox(…

In [9]:
# A new Dictionary with the actual class names, rather than class numbers
newDict = ee.Dictionary({
    1: 'Water',
    2: 'Evergreen Needleleaf Forest',
    3: 'Evergreen Broadleaf Forest' ,
    4: 'Deciduous Needleleaf Forest',
    5: 'Deciduous Broadleaf Forest',
    6: 'Mixed Forests',
    7: 'Closed Shrublands',
    8: 'Open Shrublands',
    9: 'Woody Savannas',
    10: 'Savannas',
    11: 'Grasslands',
    12: 'Permanent Wetlands',
    13: 'Croplands',
    14: 'Urban and Built-Up',
    15: 'Cropland/Natural Vegetation Mosaic',
    16: 'Snow and Ice',
    17: 'Barren or Sparsely Vegetated'
})

#pp.pprint(newDict.getInfo())

{'1': 'Water',
 '10': 'Savannas',
 '11': 'Grasslands',
 '12': 'Permanent Wetlands',
 '13': 'Croplands',
 '14': 'Urban and Built-Up',
 '15': 'Cropland/Natural Vegetation Mosaic',
 '16': 'Snow and Ice',
 '17': 'Barren or Sparsely Vegetated',
 '2': 'Evergreen Needleleaf Forest',
 '3': 'Evergreen Broadleaf Forest',
 '4': 'Deciduous Needleleaf Forest',
 '5': 'Deciduous Broadleaf Forest',
 '6': 'Mixed Forests',
 '7': 'Closed Shrublands',
 '8': 'Open Shrublands',
 '9': 'Woody Savannas'}


In [10]:
def RenameClasses(areaDict,classNumber):
    def GetNewClass(key,val):
        newClass = newDict.get(classNumber)
        return newClass

    newClassDict = areaDict.map(GetNewClass)
    newClassName = newClassDict.get('class')

    return newClassName

In [11]:
# function to get class names and get total area as a List of List
# Note: dictionary key must be of type ‘string’. Our keys are class numbers, use 
# the format() method to convert the number to string
def GetClassAreaLists(item):
    areaDict = ee.Dictionary(item)
    
    classNumber = ee.Number(areaDict.get('class')).format()
    area = ee.Number(areaDict.get('sum')).divide(1e6).round()
       
    newClassName = RenameClasses(areaDict,classNumber)
    
    return ee.List([newClassName, area])  # returns a nested list   


In [12]:
# function to compute area covered by each class
def ComputeAreaPerClass(areaImage,feature):
    areas = areaImage.reduceRegion( **{
        'reducer': ee.Reducer.sum().group( **{
            'groupField': 1,
            'groupName': 'class',
        }),
        'geometry': feature.geometry(),
        'scale': 500,
        'maxPixels': 1e10
    })
            
    return areas

In [13]:
# get the breakup of these classes by each district
def CalculateClassArea(feature):
    areaImage = ee.Image.pixelArea().addBands(classified)
    areas = ComputeAreaPerClass(areaImage,feature)    
    classAreas = ee.List(areas.get('groups'))
    classAreaLists = classAreas.map(GetClassAreaLists)       
    result = ee.Dictionary(classAreaLists.flatten())   
       
    # The result dictionary has area for all the classes
    # Add the district name to the dictionary and create a feature
    district = feature.get('ADM2_NAME')
    district_name = result.set('District', district)   
    
    return ee.Feature(feature.geometry(),district_name)#feature.geometry(), )

In [14]:
districtAreas_fc = kerala.map(CalculateClassArea)
#pp.pprint(districtAreas_fc.first().getInfo())

{'geometry': {'geometries': [{'coordinates': [...], 'type': 'LineString'},
                             {'coordinates': [...], 'type': 'LineString'},
                             {'coordinates': [...], 'type': 'LineString'},
                             {'coordinates': [...], 'type': 'LineString'},
                             {'coordinates': [...], 'type': 'LineString'},
                             {'coordinates': [...], 'type': 'LineString'},
                             {'coordinates': [...], 'type': 'LineString'},
                             {'coordinates': [...], 'type': 'Polygon'}],
              'type': 'GeometryCollection'},
 'id': '00010000000000000e0e',
 'properties': {'Barren or Sparsely Vegetated': 24,
                'Croplands': 8,
                'District': 'Kannur',
                'Evergreen Needleleaf Forest': 276,
                'Grasslands': 24,
                'Open Shrublands': 2081,
                'Permanent Wetlands': 6,
                'Savannas': 23,
    

In [15]:
new_classes = newDict.values()
#print(new_classes.getInfo())

outputFields = ee.List(['District']).cat(new_classes).getInfo()
#print(outputFields)

['District', 'Water', 'Savannas', 'Grasslands', 'Permanent Wetlands', 'Croplands', 'Urban and Built-Up', 'Cropland/Natural Vegetation Mosaic', 'Snow and Ice', 'Barren or Sparsely Vegetated', 'Evergreen Needleleaf Forest', 'Evergreen Broadleaf Forest', 'Deciduous Needleleaf Forest', 'Deciduous Broadleaf Forest', 'Mixed Forests', 'Closed Shrublands', 'Open Shrublands', 'Woody Savannas']


In [ ]:
 task = ee.batch.Export.table.toDrive( **{
    'collection': districtAreas_fc, 
    'folder': 'IWMI', 
    'description':'Kerala_class_area_by_district', 
    'fileNamePrefix': 'Kerala_',
    'fileFormat':'CSV', 
    'selectors': outputFields
 })
task.start()

while task.active():
  print('Exporting task id: {}.'.format(task.id))
  time.sleep(5)
    
#pp.pprint(task.status())

print("\n",'Exporting ', task.status().get('state'))


Make a DataFrame

In [37]:
def ToDict(fc):
    property_names = fc.first().propertyNames()
    property_value_lists = fc.reduceColumns(   # make a list for each feature (.repeat explicitly for each feature)
                                 reducer=ee.Reducer.toList().repeat(property_names.size()),
                                 selectors=property_names) \
                              .get('list')

    return ee.Dictionary.fromLists(property_names, property_value_lists)

districtAreas_dict = ToDict(districtAreas_fc).getInfo()
#pp.pprint(districtAreas_dict)

pop=districtAreas_dict.pop('system:index')

In [38]:
df = pd.DataFrame.from_dict(districtAreas_dict, orient='index').T

# index from 1
df.index = np.arange( 1, len(df) + 1)

# District as first column
cols = ['District']  + [col for col in df if col != 'District']
df = df[cols]

df

,District,Barren or Sparsely Vegetated,Croplands,Evergreen Needleleaf Forest,Grasslands,Open Shrublands,Permanent Wetlands,Savannas,Urban and Built-Up,Woody Savannas
1,Kannur,24,8,276,24,2081,6,23,291,67
2,Ernakulam,71,117,87,82,1134,8,2,692,108
3,Idukki,15,2,1764,7,2997,2,59,8,133
4,Kasaragod,19,1,123,14,881,5,12,689,219
5,Kottayam,43,10,62,77,1524,66,26,302,23
6,Kozhikode,19,12,291,38,1619,16,22,258,21
7,Malappuram,11,13,475,37,1160,764,6,1588,179
8,Palakkad,9,30,818,5,891,13,27,1592,187
9,Pattanamtitta,2,7,1226,1,1118,0,2,199,33
10,Kollam,24,24,599,48,1240,88,2,323,88
